# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vacaweather_df=pd.read_csv (r"output_data/cities.csv")
print(vacaweather_df)

     Unnamed: 0              City      Lat       Lng  Max Temp(f)  Humidity  \
0             0    Port Elizabeth -33.9180   25.5701        49.32        88   
1             1        Nanortalik  60.1432  -45.2371        45.36        76   
2             2           Rikitea -23.1203 -134.9692        73.11        74   
3             3         Kavaratti  10.5669   72.6420        77.22        88   
4             4       Tuktoyaktuk  69.4541 -133.0374        53.60        87   
..          ...               ...      ...       ...          ...       ...   
546         546     Arroyo Grande  35.1186 -120.5907        88.54        65   
547         547           Bubaque  11.2833  -15.8333        79.18        82   
548         548          Tessalit  20.1986    1.0114        93.34        26   
549         549  Vigía del Fuerte   6.5789  -76.8863        74.50        97   
550         550  Highland Springs  37.5460  -77.3278        81.32        92   

     Cloudiness  Wind Speed Country        Date  
0

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(g_key)

In [5]:
locations = vacaweather_df[["Lat","Lng"]]
humidity = vacaweather_df["Humidity"]

In [6]:
locations

,Lat,Lng
0,-33.9180,25.5701
1,60.1432,-45.2371
2,-23.1203,-134.9692
3,10.5669,72.6420
4,69.4541,-133.0374
...,...,...
546,35.1186,-120.5907
547,11.2833,-15.8333
548,20.1986,1.0114
549,6.5789,-76.8863


In [7]:
fig=gmaps.figure()
# df = gmaps.datasets.vacaweather_df()
heat_layer=gmaps.heatmap_layer(
        locations, weights=humidity, 
        dissipating=False, 
        max_intensity=100, 
        point_radius=1
        )
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
perftemp_df = vacaweather_df.loc[(vacaweather_df["Max Temp(f)"]>70) & (vacaweather_df["Max Temp(f)"]<80)]
# perftemp_df.count()
perfwind_df = perftemp_df.loc[(vacaweather_df["Wind Speed"]< 10)]
# perfwind_df.count()
# For Humidity; If I keep it at perfect humidity I get very low results
perfwind_df = perftemp_df.loc[(vacaweather_df["Humidity"]< 40)]
# perfwind_df.count()
perfvacaweather_df = perfwind_df[(vacaweather_df["Cloudiness"]<25)]
perfvacaweather_df




/var/folders/v0/fgqm4drj0fv_rywhmh177t240000gp/T/ipykernel_50378/2517444999.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perfvacaweather_df = perfwind_df[(vacaweather_df["Cloudiness"]<25)]


,Unnamed: 0,City,Lat,Lng,Max Temp(f),Humidity,Cloudiness,Wind Speed,Country,Date
30,30,Tāybād,34.7400,60.7756,76.32,33,0,18.59,IR,1659401010
157,157,Norman Wells,65.2820,-126.8329,75.24,38,20,9.22,CA,1659400606
201,201,Fuerte Olimpo,-21.0415,-57.8738,76.80,35,0,5.17,PY,1659401062
374,374,Mount Isa,-20.7333,139.5000,76.77,33,0,11.50,AU,1659401050
432,432,Yangi Marg`ilon,40.4272,71.7189,75.74,38,21,2.19,UZ,1659400574
439,439,Campina Verde,-19.5356,-49.4864,73.26,35,17,4.32,BR,1659401154
447,447,Oshnavīyeh,37.0397,45.0983,70.90,28,0,2.71,IR,1659401156
454,454,Bingöl,38.8847,40.4939,70.11,28,0,4.56,TR,1659401159


In [9]:
perfvacaweather_df


,Unnamed: 0,City,Lat,Lng,Max Temp(f),Humidity,Cloudiness,Wind Speed,Country,Date
30,30,Tāybād,34.7400,60.7756,76.32,33,0,18.59,IR,1659401010
157,157,Norman Wells,65.2820,-126.8329,75.24,38,20,9.22,CA,1659400606
201,201,Fuerte Olimpo,-21.0415,-57.8738,76.80,35,0,5.17,PY,1659401062
374,374,Mount Isa,-20.7333,139.5000,76.77,33,0,11.50,AU,1659401050
432,432,Yangi Marg`ilon,40.4272,71.7189,75.74,38,21,2.19,UZ,1659400574
439,439,Campina Verde,-19.5356,-49.4864,73.26,35,17,4.32,BR,1659401154
447,447,Oshnavīyeh,37.0397,45.0983,70.90,28,0,2.71,IR,1659401156
454,454,Bingöl,38.8847,40.4939,70.11,28,0,4.56,TR,1659401159


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Insert column for "Hotels"
hotel_df=perfvacaweather_df[["City","Country","Lat","Lng", "Max Temp(f)", "Humidity","Cloudiness","Wind Speed","Date"]]
hotel_df["Hotel Name"] =""
display(hotel_df)
# hotel_df = perfvacaweather_df.insert(0,"Hotel Name", [max],allow_duplicates=False)
# print(hotel_df)

,City,Country,Lat,Lng,Max Temp(f),Humidity,Cloudiness,Wind Speed,Date,Hotel Name
30,Tāybād,IR,34.7400,60.7756,76.32,33,0,18.59,1659401010,
157,Norman Wells,CA,65.2820,-126.8329,75.24,38,20,9.22,1659400606,
201,Fuerte Olimpo,PY,-21.0415,-57.8738,76.80,35,0,5.17,1659401062,
374,Mount Isa,AU,-20.7333,139.5000,76.77,33,0,11.50,1659401050,
432,Yangi Marg`ilon,UZ,40.4272,71.7189,75.74,38,21,2.19,1659400574,
439,Campina Verde,BR,-19.5356,-49.4864,73.26,35,17,4.32,1659401154,
447,Oshnavīyeh,IR,37.0397,45.0983,70.90,28,0,2.71,1659401156,
454,Bingöl,TR,38.8847,40.4939,70.11,28,0,4.56,1659401159,


In [11]:
# geocoordinates
target_type="lodging"
Radius = 5000

params= {
        
        "radius": Radius,
        "types": "lodging",
        "key": g_key,
}
for index, row in hotel_df.iterrows():
        lat= row["Lat"]
        lng= row["Lng"]
        params ["location"] = f"{lat},{lng}"

        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        hotels = requests.get(base_url, params=params).json()
#         print(hotels)
        try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
            
        except (KeyError, IndexError):
            print("Missing Hotel in Area")
            print("------------------")
hotel_df

,City,Country,Lat,Lng,Max Temp(f),Humidity,Cloudiness,Wind Speed,Date,Hotel Name
30,Tāybād,IR,34.7400,60.7756,76.32,33,0,18.59,1659401010,لی پوش jean center
157,Norman Wells,CA,65.2820,-126.8329,75.24,38,20,9.22,1659400606,Heritage Hotel
201,Fuerte Olimpo,PY,-21.0415,-57.8738,76.80,35,0,5.17,1659401062,Hospedaje DAMI
374,Mount Isa,AU,-20.7333,139.5000,76.77,33,0,11.50,1659401050,ibis Styles Mt Isa Verona
432,Yangi Marg`ilon,UZ,40.4272,71.7189,75.74,38,21,2.19,1659400574,Ikathouse
439,Campina Verde,BR,-19.5356,-49.4864,73.26,35,17,4.32,1659401154,Hotel Lino
447,Oshnavīyeh,IR,37.0397,45.0983,70.90,28,0,2.71,1659401156,Aram Qurbani
454,Bingöl,TR,38.8847,40.4939,70.11,28,0,4.56,1659401159,Vali Konağı


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color ="rgba(0,150,0, 0.4)",
    stroke_color="rgba(0,0,150,0.4)", scale=2,
#     info_box_content=[f"Hotel: {Hotel Name}" for hotel in hotel_df]
)

# Display figure
fig = gmaps.figure()
fig.add_layer(hotel_layer)
                      
fig

Figure(layout=FigureLayout(height='420px'))